In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd

In [ ]:
train=pd.read_csv('/content/drive/MyDrive/Data Science/train.csv')

In [ ]:
train.head()

,id,comment_text,malignant,highly_malignant,rude,threat,abuse,loathe
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0


In [ ]:
train.shape

(159571, 8)

In [ ]:
train.isnull().sum()

id                  0
comment_text        0
malignant           0
highly_malignant    0
rude                0
threat              0
abuse               0
loathe              0
dtype: int64

In [ ]:
# There are no null values thankfully

In [ ]:
sentiment = train['comment_text'].values
y_list = ["malignant","highly_malignant", "rude", "threat", "abuse", "loathe"]
y = train[y_list].values

In [ ]:
type(sentiment)

numpy.ndarray

In [ ]:
type(y)

numpy.ndarray

In [ ]:
sentiment.shape

(159571,)

In [ ]:
y.shape

(159571, 6)

In [ ]:
print(sentiment[15])

"

Juelz Santanas Age

In 2002, Juelz Santana was 18 years old, then came February 18th, which makes Juelz turn 19 making songs with The Diplomats. The third neff to be signed to Cam's label under Roc A Fella. In 2003, he was 20 years old coming out with his own singles ""Santana's Town"" and ""Down"". So yes, he is born in 1983. He really is, how could he be older then Lloyd Banks? And how could he be 22 when his birthday passed? The homie neff is 23 years old. 1983 - 2006 (Juelz death, god forbid if your thinking about that) equals 23. Go to your caculator and stop changing his year of birth. My god."


In [ ]:
len(sentiment[15])

610

In [ ]:
print(y[60])

[0 0 0 0 0 0]


In [ ]:
from keras.preprocessing.text import Tokenizer
tokenizer = Tokenizer(num_words=25000)
tokenizer.fit_on_texts(list(sentiment))


In [ ]:
from keras.preprocessing import text, sequence

seq = tokenizer.texts_to_sequences(sentiment)
pad = sequence.pad_sequences(seq, maxlen=300)

In [ ]:
type(seq)

list

In [ ]:
len(seq[0])

47

In [ ]:
type(pad)

numpy.ndarray

In [ ]:
pad.shape

(159571, 300)

In [ ]:
test=pd.read_csv('/content/drive/MyDrive/Data Science/test.csv')
test.head()

,id,comment_text
0,00001cee341fdb12,Yo bitch Ja Rule is more succesful then you'll...
1,0000247867823ef7,== From RfC == \n\n The title is fine as it is...
2,00013b17ad220c46,""" \n\n == Sources == \n\n * Zawe Ashton on Lap..."
3,00017563c3f7919a,":If you have a look back at the source, the in..."
4,00017695ad8997eb,I don't anonymously edit articles at all.


In [ ]:
test.shape

(153164, 2)

In [ ]:
test = test['comment_text'].values
test_seq = tokenizer.texts_to_sequences(test)
test_pad = sequence.pad_sequences(test_seq, maxlen=300)

In [ ]:
from keras.layers import Dense, Input, LSTM, Embedding, Dropout, Activation
from keras.layers import Bidirectional, GlobalMaxPool1D
from keras.models import Model
from keras import initializers, regularizers, constraints, optimizers, layers


def model_add():
    inputs = Input(shape=(300, ))
    x = Embedding(25000, 128)(inputs)
    x = Bidirectional(LSTM(50))(x)
    x = Dropout(0.1)(x)
    x = Dense(50, activation="relu")(x)
    x = Dropout(0.1)(x)
    outputs = Dense(6, activation="sigmoid")(x)
    model = Model(inputs=inputs, outputs=outputs)
    model.compile(loss='binary_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy','Precision','Recall'])
    return model
model = model_add()
print(model.summary())

Model: "functional_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         [(None, 300)]             0         
_________________________________________________________________
embedding_2 (Embedding)      (None, 300, 128)          3200000   
_________________________________________________________________
bidirectional_2 (Bidirection (None, 100)               71600     
_________________________________________________________________
dropout_4 (Dropout)          (None, 100)               0         
_________________________________________________________________
dense_4 (Dense)              (None, 50)                5050      
_________________________________________________________________
dropout_5 (Dropout)          (None, 50)                0         
_________________________________________________________________
dense_5 (Dense)              (None, 6)                

In [ ]:
from keras.callbacks import EarlyStopping, ModelCheckpoint
early = EarlyStopping(monitor="val_loss", mode="min", patience=2)
model.fit(pad, y, batch_size=32, epochs=3, validation_split=0.15, callbacks=early)

Epoch 1/3
4239/4239 [==============================] - 241s 57ms/step - loss: 0.0669 - accuracy: 0.9550 - precision: 0.7832 - recall: 0.5664 - val_loss: 0.0523 - val_accuracy: 0.9944 - val_precision: 0.7758 - val_recall: 0.6841
Epoch 2/3
4239/4239 [==============================] - 241s 57ms/step - loss: 0.0476 - accuracy: 0.9930 - precision: 0.8213 - recall: 0.6665 - val_loss: 0.0488 - val_accuracy: 0.9944 - val_precision: 0.8347 - val_recall: 0.6331
Epoch 3/3
4239/4239 [==============================] - 241s 57ms/step - loss: 0.0416 - accuracy: 0.9927 - precision: 0.8335 - recall: 0.7089 - val_loss: 0.0473 - val_accuracy: 0.9944 - val_precision: 0.8178 - val_recall: 0.6645


In [ ]:
# The model is performing well and there is no over fitting

In [ ]:
y_test = model.predict([test_pad], batch_size=1024, verbose=1)

150/150 [==============================] - 9s 62ms/step


In [ ]:
type(y_test)

numpy.ndarray

In [ ]:
y_test.shape

(153164, 6)

In [ ]:
y_test[0]

array([0.99415094, 0.17568958, 0.9159673 , 0.07157331, 0.83640176,
       0.17543791], dtype=float32)

In [ ]:
# Now, we have probabilities of our comment belonging to each of the 6 classes.
# We can thus set as threshold and classify the comment belonging to that class if the probability value crosses that threshold for that particular class.